In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.callbacks import EarlyStopping
import pickle
import numpy as np

ModuleNotFoundError: No module named 'tensorflow.python.keras.preprocessing'

In [33]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender_h.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo_h.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler_h.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [38]:
## Define a functiojn to create the model and try different parameters(KerasClassifier)
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    
    # Ensure model is compiled within the function and returned
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy']) 

    return model

In [42]:
## Create a Keras classifier
model = KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [43]:
## Define the grid search parameters 
param_grid = {
    'neurons' : [16, 32, 64, 128],
    'layers' : [1, 2],
    'epochs' : [50, 100]
}

In [44]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Users/abhishekkumar/Desktop/NLP/UdemyNLP/ANNPracticals/myenv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/abhishekkumar/Desktop/NLP/UdemyNLP/ANNPracticals/myenv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/abhishekkumar/Desktop/NLP/UdemyNLP/ANNPracticals/myenv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/abhishekkumar/Desktop/NLP/UdemyNLP/ANNPracticals/myenv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``b

ValueError: 
All the 48 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/abhishekkumar/Desktop/NLP/UdemyNLP/ANNPracticals/myenv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/abhishekkumar/Desktop/NLP/UdemyNLP/ANNPracticals/myenv/lib/python3.11/site-packages/scikeras/wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/Users/abhishekkumar/Desktop/NLP/UdemyNLP/ANNPracticals/myenv/lib/python3.11/site-packages/scikeras/wrappers.py", line 770, in fit
    self._fit(
  File "/Users/abhishekkumar/Desktop/NLP/UdemyNLP/ANNPracticals/myenv/